# Применение инструментов Hugging face и предобученных моделей

### Вариант 1:
 Вам нужно создать искусственные данные для тестирования и/или обучения чат-бота. По заданному предложению/утверждению/команде создать набор расширенных предложений/утверждений/команд с приблизительно тем же смыслом. Пример:

> After your workout, remember to focus on maintaining a good water balance.

похожие команды:

> Remember to drink enough water to restore and maintain your body's hydration after your cardio training.

>Please don't forget to maintain water balance after workout.

Предлагается решить упрощенную версию данной задачи с применением общедоступных "маленьких". 
В репозитории Hugging Face есть большое количество предобученных моделей для [casual](https://huggingface.co/models?pipeline_tag=text-generation) и [masked](https://huggingface.co/models?pipeline_tag=fill-mask) языкового моделирования.  Также для валидации можно использовать [sentence-transformers](https://huggingface.co/sentence-transformers). Выбрать нужно модели, которые можно запускать на CPU.

Пример использования masked LM:

```python
import torch
from transformers import AutoModelForMaskedLM, PreTrainedTokenizer

# загружается токенайзер
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
# загружается модель
model = AutoModelForMaskedLM.from_pretrained("distilroberta-base")

# предложение и замаскированным токеном
sequence = f"My name is {tokenizer.mask_token}."

# результат токенизации
input_ids = tokenizer.encode(sequence, return_tensors="pt")
# применение модели
result = model(input_ids=input_ids)

# индекс замаскированного токена (NB может не совпадать с номером слова)
mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

# самый вероятный токен 
print(tokenizer.decode(result.logits[:, mask_token_index].argmax()))
```

или через [pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines)

```python
from transformers import pipeline

pipe = pipeline("fill-mask", model="distilroberta-base")

pipe("My name is <mask>.")
```

Casual LM через pipeline:

```python
from transformers import pipeline, set_seed

generator = pipeline('text-generation', model='gpt2')

generator("Hello", max_length=10, num_return_sequences=5)
```

Один наивных способов решения задачи без дополнительного обучения - замаскировать, или вставить в исходную команду замаскированный токен, или обрезать часть команды и применить языковую модель. Результат можно валидировать с помощью [sentence-transformers](https://huggingface.co/sentence-transformers). 


In [1]:
import torch
import numpy as np
import re
from transformers import AutoModelForMaskedLM, AutoTokenizer

# загружается токенайзер
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
# загружается модель
model = AutoModelForMaskedLM.from_pretrained("distilroberta-base")

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
# предложение с замаскированным токеном
sequence = f"My name is {tokenizer.mask_token}."

# результат токенизации
input_ids = tokenizer.encode(sequence, return_tensors="pt")
# применение модели
result = model(input_ids=input_ids)

In [3]:
# индекс замаскированного токена (NB может не совпадать с номером слова)
mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

# самый вероятный токен 
print(tokenizer.decode(result.logits[:, mask_token_index].argmax()))

 Jason


In [4]:
# первые N самых вероятных слов
N = 5
[tokenizer.decode(x) for x in result.logits[0, mask_token_index.tolist()[0]].argsort(descending=True).tolist()[:N]]

[' Jason', ' Alex', ' Chris', ' David', ' Brian']

In [5]:
def get_N_most_possible_tokens(sequence, N=5):
    # результат токенизации
    input_ids = tokenizer.encode(sequence, return_tensors="pt")
    # применение модели
    result = model(input_ids=input_ids)
    # индекс замаскированного токена (NB может не совпадать с номером слова)
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]    
    return [tokenizer.decode(x) for x in result.logits[0, mask_token_index.tolist()[0]].argsort(descending=True).tolist()[:N]]

Валидация с помощью sentence-transformers

In [6]:
from sentence_transformers import SentenceTransformer, util

# Загрузка модели sentence-transformers
sbert_model = SentenceTransformer("all-MiniLM-L12-v2")

def cos_score(sentence: str, new_sentence: str) -> float:
    # Кодирование исходного и новых предложений
    sentence_embeddings = sbert_model.encode([sentence], convert_to_tensor=True)
    new_sentence_embeddings = sbert_model.encode([new_sentence], convert_to_tensor=True)

    # Расчет косинусного расстояния между векторами
    cosine_scores = util.pytorch_cos_sim(sentence_embeddings, new_sentence_embeddings)

    return cosine_scores.item()


def check_coincidence(command, new_command):
    return ' '.join(re.sub(r'[^\w\s]', '', command).lower().split(' ')) != ' '.join(new_command.lower().split(' '))


def filtering(command, commands, th=0.9):
    result = []
    for new_command in commands:
        if cos_score(command, new_command) > th and check_coincidence(command, new_command):
            result.append(new_command)
    return list(set(result))


In [7]:
sentence1 = "My name is Gosha."
new_sentence1 = "Gosha is my name"
print(cos_score(sentence1, new_sentence1))

sentence2 = "My name is Gosha."
new_sentence2 = "Jason is my name"
print(cos_score(sentence2, new_sentence2))

0.9411747455596924
0.4705086052417755


In [21]:
def using_maskLM(command, insert_mask=True):

    cleaned_command = re.sub(r'[^\w\s]', '', command.lower())
    command_words = cleaned_command.split(' ')

    low, high = 1, 3
    masks = np.random.randint(low, high)

    commands = []
    
    for _ in range(masks):
        if insert_mask:
            index = np.random.choice(range(0, len(command_words) + 1))
            command_words.insert(index, tokenizer.mask_token)
        else:
            index = np.random.choice(range(0, len(command_words)))
            command_words = [command_words[i] if i != index else tokenizer.mask_token for i in range(len(command_words))]

        masked_command = " ".join(command_words)
        variants = get_N_most_possible_tokens(masked_command, 5)
        good_variants =  [tok.strip() for tok in variants if len(tok) > 1]
        for good in good_variants:
            new_command = masked_command.replace(tokenizer.mask_token, good)
            new_command = ' '.join(new_command.split())
            if len(commands) == 0:
                commands.append(new_command)
            elif all([check_coincidence(new_command, command) for command in commands]):
                commands.append(new_command)

    return commands


def generate_similar_commands(command, num=10):
    commands = []
 
    for _ in range(num // 2):
        commands.extend(using_maskLM(command, insert_mask=True))

    for _ in range(num // 2, num):
        commands.extend(using_maskLM(command, insert_mask=False))

    return commands

np.random.seed(0)
command = "After your workout, remember to focus on maintaining a good water balance."
new_commands = generate_similar_commands(command, 20)

res = filtering(command, new_commands, th=0.5)
print(len(res))
display(res)


105


['after your workout remember to focus only on only maintaining a good water balance',
 'after your workout remember to focus on consistently maintaining a good water balance',
 'after your workout remember to really focus on maintaining a good water balance',
 'after your workout remember how to focus on maintaining a good water balance',
 'after your workout and remember to focus on maintaining a good water balance',
 'During your workout remember to focus on maintaining a good water balance',
 'after your workout remember the focus on maintaining a good the balance',
 'after your workout and remember to focus and on maintaining a good water balance',
 'after your workout remember to focus instead on maintaining a good water balance',
 'after performing your workout remember to focus on maintaining performing a good water balance',
 'after your workout remember to focus on maintaining your your water balance',
 'after your workout always remember always to focus on maintaining a good

Теперь попробуем изменять несколько раз одну и ту же стрчоку

In [9]:
def using_maskLM(command):
    
    insert_mask = np.random.choice([False, True], p=(0.3, 0.7))

    cleaned_command = re.sub(r'[^\w\s]', '', command.lower())
    command_words = cleaned_command.split(' ')

    if insert_mask:
        index = np.random.choice(range(0, len(command_words) + 1))
        command_words.insert(index, tokenizer.mask_token)
    else:
        index = np.random.choice(range(0, len(command_words)))
        command_words = [command_words[i] if i != index else tokenizer.mask_token for i in range(len(command_words))]

    masked_command = " ".join(command_words)
    variants = get_N_most_possible_tokens(masked_command, 5)
    good_variants =  [tok.strip() for tok in variants if len(re.sub(r'[^\w\s]', '', tok)) > 1]
    if len(good_variants) == 0:
        good = ""
    else:
        good = np.random.choice(good_variants)
    
    new_command = masked_command.replace(tokenizer.mask_token, good)
    new_command = ' '.join(new_command.split())
    return new_command 


def generate_similar_commands(command, num=10):
    commands = []

    K = len(command.split(' ')) + 1

    for _ in range(num):
        new_command = command
        for _ in range(K):
            new_command = using_maskLM(new_command)

            if len(commands) == 0:
                commands.append(new_command)
            elif all([check_coincidence(new_command, command) for command in commands]):
                commands.append(new_command)

    return commands

np.random.seed(0)
command = "After your workout, remember to focus on maintaining a good water balance."
new_commands = generate_similar_commands(command, 20)

res = filtering(command, new_commands, th=0.5)
print(len(res))
display(res)


228


['after your workout remember to instead focus on maintaining maintaining a good water balance',
 'after your workout remember how to focus on yourself while maintaining a good water balance',
 'after your workout remember try to focus properly on yourself while retaining such a good body water balance',
 'after your next workout and remember to focus on maintaining maintaining for a good water balance',
 'after successfully performing your first workout remember to focus attention properly balance maintaining a good water balance',
 'day after your workout so remember to really focus on physically and maintaining a clean water balance and hygiene',
 'after your meal and then remember to focus on your always having a good water balance',
 'after your workout but remember always to focus instead instead on exercising just maintaining maintaining a very good water temperature',
 'after your workout but remember always to focus instead instead on just maintaining maintaining a very good w

Результат получше, можно ещё параметрами поуправлять

Теперь черезе pipelines

In [10]:
from transformers import pipeline

pipe = pipeline("fill-mask", model="distilroberta-base")


def get_new_command(command):
    cleaned_command = re.sub(r'[^\w\s]', '', command.lower())
    command_words = cleaned_command.split(' ')

    insert_mask = np.random.choice([False, True], p=(0.6, 0.4))
    if insert_mask:
        index = np.random.choice(range(0, len(command_words) + 1))
        command_words.insert(index, tokenizer.mask_token)
    else:
        index = np.random.choice(range(0, len(command_words)))
        command_words = [command_words[i] if i != index else tokenizer.mask_token for i in range(len(command_words))]

    masked_command = " ".join(command_words) 
    new_command = np.random.choice(pipe(masked_command))["sequence"]
    return new_command


def generate_similar_commands_with_pipe(command, num=5):
    commands = []

    K = len(command.split(' ')) + 1

    for _ in range(num):
        new_command = command
        for _ in range(25):
            new_command = get_new_command(new_command)

            if len(commands) == 0:
                commands.append(new_command)
            elif all([check_coincidence(new_command, command) for command in commands]):
                commands.append(new_command)
    return commands

np.random.seed(2)
command = "After your workout, remember to focus on maintaining a good water balance."
new_commands = generate_similar_commands_with_pipe(command, 5)

res = filtering(command, new_commands, th=0.5)
print(len(res))
display(res)



Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


97


['afternoon workout remember how not to really focus more on energy maintaining just some liquid oxygen',
 'after your workout remember yourself to focus on maintaining some liquid water retention',
 'after finishing your workout remember to constantly strive for maintaining at a good water pressure',
 'afternoon workout remember not to really focus on maintaining some maximal oxygen retention',
 'During your workout remember to focus on maintaining a good water balance',
 'during this workout remember your focus on maintaining consistently good water quality',
 'after completing almost every workout i remember your focus on maintaining the consistently consistent good vaping & vape juice quality',
 ' during your workout remember to work on maintaining a good muscle balance',
 'afternoon workout remember how not to just focus more on energy maintaining with some liquid oxygen',
 'As always during your workout remember to work on maintaining for a good muscle balance',
 'items needed du

Теперь через Casual LM через pipeline

In [11]:
from transformers import pipeline, set_seed

generator = pipeline('text-generation', model='gpt2')

generator("After your workout, remember to focus on maintaining a good water balance.", max_length=20, num_return_sequences=1)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'After your workout, remember to focus on maintaining a good water balance. This means avoiding any foods that'

In [23]:
def get_new_command_gpt2(command):
    cleaned_command = re.sub(r'[^\w\s]', '', command.lower())
    command_words = cleaned_command.split(' ')
    index = np.random.choice(range(0, len(command_words)))
    new_command = " ".join(command_words[:index])
    new_command = generator(new_command, max_length=len(command_words) + 5, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    new_command = re.sub(r'[^\w\s]', '', new_command.lower()).split(" ")[:index + 1]
    # убираем возможные /n
    new_command = " ".join(new_command).split()
    new_command.extend(command_words[index:])
    return " ".join(new_command)


def generate_similar_commands_with_gpt2(command, num=5):
    commands = []

    K = len(command.split(' ')) + 1

    for _ in range(num):
        new_command = command
        for _ in range(2):
            new_command = get_new_command_gpt2(new_command)

            if len(commands) == 0:
                commands.append(new_command)
            elif all([check_coincidence(new_command, command) for command in commands]):
                commands.append(new_command)
    return commands


command = "After your workout, remember to focus on maintaining a good water balance."
new_commands = generate_similar_commands_with_gpt2(command, 30)

res = filtering(command, new_commands, th=0.5)
print(len(res))
display(res)



58


['by after your workout remember to focus on maintaining a good water balance',
 'after your friend workout ill remember to focus on maintaining a good water balance',
 'after your workout remember to have focus on maintaining a good water balance',
 'after your workout was remember to hit focus on maintaining a good water balance',
 'the after your death workout remember to focus on maintaining a good water balance',
 'after your workout remember that to to focus on maintaining a good water balance',
 'after your workout remember to focus on the maintaining a good water balance',
 'after your workout remember to have focus on maintaining a good water balance balance',
 'after your workout remember to clean focus on the maintaining a good water balance',
 'after of your workout remember to focus on on maintaining a good water balance',
 'dell after your workout remember to focus on maintaining a positive good water balance',
 'after your eyes workout remember to focus on maintaining a 

Объеденим всё вместе

In [13]:
from transformers import pipeline

pipe = pipeline("fill-mask", model="distilroberta-base")
generator = pipeline('text-generation', model='gpt2')

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
-2 % 5

3

In [25]:
def get_new_command_bert(command):
    cleaned_command = re.sub(r'[^\w\s]', '', command.lower())
    command_words = cleaned_command.split(' ')

    insert_mask = np.random.choice([False, True], p=(0.6, 0.4))
    if insert_mask:
        index = (np.random.choice(range(0, len(command_words) + 1)) - 3) % len(command_words)
        command_words.insert(index, tokenizer.mask_token)
    else:
        index = (np.random.choice(range(0, len(command_words))) - 3) % len(command_words)
        command_words = [command_words[i] if i != index else tokenizer.mask_token for i in range(len(command_words))]

    masked_command = " ".join(command_words) 
    new_command = np.random.choice(pipe(masked_command))["sequence"]
    return new_command


def get_new_command_gpt2(command):
    cleaned_command = re.sub(r'[^\w\s]', '', command.lower())
    command_words = cleaned_command.split(' ')
    index = np.random.choice(range(0, len(command_words)))
    new_command = " ".join(command_words[:index])
    new_command = generator(new_command, max_length=len(command_words) + 5, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    new_command = re.sub(r'[^\w\s]', '', new_command.lower()).split(" ")[:index + 1]
    # убираем возможные /n
    new_command = " ".join(new_command).split()
    new_command.extend(command_words[index:])
    return " ".join(new_command)


def generate_similar_commands(command, num=5, k=25, p=[0.8, 0.2]):
    commands = []

    functions = [get_new_command_bert, get_new_command_gpt2]

    for _ in range(num):
        new_command = command
        for _ in range(k):
            function = np.random.choice(functions, p=p)
            
            new_command = function(new_command)

            if len(commands) == 0:
                commands.append(new_command)
            elif all([check_coincidence(new_command, command) for command in commands]):
                commands.append(new_command)
    return commands


In [26]:
command = "After your workout, remember to focus on maintaining a good water balance."
new_commands = generate_similar_commands(command, num=50, k=13, p=[0.8, 0.2])

res = filtering(command, new_commands, th=0.5)
print(len(res))
display(res)

C:\Users\Egor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\utils.py:1363: UserWarning: Input length of input_ids is 17, but `max_length` is set to 17. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


583


['after your workout remember to focus exclusively on the maintaining a good water balance',
 'make your workout attempt to focus on maintaining a very good drinking tap water intake',
 'after your workout remember yourself i to keep your focus on maintaining a good physical blood water body balance',
 'after your first workout to really focus on maintaining such a good water balance',
 'eat after each workout but remember always check focus on maintaining a good water balance',
 'day throughout your diet remember to consume not focus exclusively on maintaining yourself a very good water balance',
 'after finishing a workout remember to focus on on maintaining a good water balance',
 'eat after each workout but remember to check focus on maintaining a good water balance',
 'after your workout remember yourself to keep on maintaining a good water level balance',
 'after your workout if you remember to focus on having a good resting water temperature',
 'Improve your workout if you remem